# 🏭 Pipeline de Forecasting por Producto usando AutoGluon

En este notebook vamos a entrenar y validar modelos de series temporales por producto (`product_id`), prediciendo las toneladas vendidas (`tn`) a 2 períodos hacia adelante. Se incluyen variables adicionales y se evalúa el uso de AutoGluon para forecasting multivariante.

- **Entrada:** parquet `"dataset_product_periodo.parquet"`
- **Objetivo:** predecir `tn` para cada producto en el período `202002` (`M+2` respecto al último dato disponible).
- **Salida:** CSV con columnas `product_id` y `tn` (pronosticado para `202002`).


## 🔢 Celda de código – Carga de librerías

In [3]:
# Carga de librerías principales
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesPredictor
import matplotlib.pyplot as plt


## 📦 Carga y preprocesamiento de datos

Levantamos el archivo parquet generado en la etapa anterior, verificamos la estructura y visualizamos los primeros registros.


In [81]:
ruta_parquet = "C:/Developer/Laboratorio_III/data/dataset_product_periodo.parquet"
df = pd.read_parquet(ruta_parquet)
df.head()


,product_id,periodo,tn_total,clientes_positivos,cat1,cat2,cat3,brand,sku_size,descripcion,fecha,mm-yyyy,quarter
0,20001,201701,934.77222,186,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-01-01,01-2017,2017Q1
1,20001,201702,798.01620,185,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-02-01,02-2017,2017Q1
2,20001,201703,1303.35771,188,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-03-01,03-2017,2017Q1
3,20001,201704,1069.96130,104,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-04-01,04-2017,2017Q2
4,20001,201705,1502.20132,238,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,2017-05-01,05-2017,2017Q2


## 🛠️ Preparación de datos para AutoGluon

AutoGluon espera los datos en formato *long* con columnas:

- `item_id` (identificador de la serie, aquí: `product_id`)
- `timestamp` (fecha o período, tipo datetime)
- `target` (variable a predecir, aquí: `tn_total`)
- Variables adicionales opcionales (features, por ejemplo: `clientes_positivos`, `cat1`, `quarter`, etc.)

Reformateamos y seleccionamos columnas.


### 📄 Celda de código – Reformateo para AutoGluon

In [86]:
# Renombrar columnas según lo esperado por AutoGluon
df_ag = df.rename(columns={
    'product_id': 'item_id',
    'fecha': 'timestamp',
    'tn_total': 'target'
})

# Convertir timestamp a datetime si es necesario
if not np.issubdtype(df_ag['timestamp'].dtype, np.datetime64):
    df_ag['timestamp'] = pd.to_datetime(df_ag['timestamp'])

# Definir las features adicionales (todas menos las esenciales)
exclude = ['item_id', 'timestamp', 'target']
features = [col for col in df_ag.columns if col not in exclude]

# Seleccionar solo las columnas necesarias
df_ag = df_ag[['item_id', 'timestamp', 'target'] + features]
df_ag = df_ag.sort_values(['item_id', 'timestamp'])
df_ag.head()
features


['periodo',
 'clientes_positivos',
 'cat1',
 'cat2',
 'cat3',
 'brand',
 'sku_size',
 'descripcion',
 'mm-yyyy',
 'quarter']

### 🏋️ Celda de código – Entrenamiento del predictor

In [ ]:
# Determinamos el horizonte de predicción
prediction_length = 2

# Directorio de trabajo para AutoGluon
output_directory = "autogluon_forecasting_product"

predictor = TimeSeriesPredictor(
    target='target',
    prediction_length=prediction_length,
    eval_metric='MASE',
    path=output_directory,
    known_covariates_names=features,  # ✅ nombre correcto y lugar correcto
    freq='M'  # 🧠 frecuencia mensual explícita
)


# Entrenar con todas las features excepto las excluidas
predictor.fit(
    train_data=df_ag,
    time_limit=3600  # ahora queda solo esto acá
)



Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Developer\Laboratorio_III\notebooks\autogluon_forecasting_product'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.21
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          14
GPU Count:          0
Memory Avail:       3.57 GB / 15.31 GB (23.3%)
Disk Space Avail:   234.59 GB / 475.95 GB (49.3%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'M',
 'hyperparameters': 'default',
 'known_covariates_names': ['periodo',
                            'clientes_positivos',
                            'cat1',
                            'cat2',
                            'cat3',
                            'brand',
                            'sku_size',
                            'descripcion',
                            'mm-yyyy',
                            'quarter'],
 '

## 🔮 Predicción para productos objetivo en período 202002

Predecimos `tn` para cada producto en el período `202002` (febrero 2020). Extraemos y guardamos el resultado en CSV.


### 📥 Celda de código – Cargar IDs desde archivo .txt con encabezado

In [40]:
# Ruta al archivo con encabezado "product_id"
ruta_ids = "C:/Developer/Laboratorio_III/data/product_id_apredecir201912.txt"

# Cargar los IDs respetando el encabezado
df_ids = pd.read_csv(ruta_ids, sep=',')  # O ajustar sep si es tabulado
product_ids_a_predecir = df_ids['product_id'].unique().tolist()

# Verificamos
print(f"✅ Se cargarán {len(product_ids_a_predecir)} productos para predicción")


✅ Se cargarán 780 productos para predicción


### 📥 Celda de código – Cargar enriquecer los datos de entrada a la predicción

In [67]:
from autogluon.timeseries import TimeSeriesDataFrame

# ----------------------------
# 1️⃣ Cargar lista de productos a predecir
# ----------------------------
ruta_ids = "C:/Developer/Laboratorio_III/data/product_id_apredecir201912.txt"
df_ids = pd.read_csv(ruta_ids, sep='\t')  # Asegurate que el separador sea correcto
df_ids.rename(columns={'product_id': 'item_id'}, inplace=True)
ids_pred = df_ids['item_id'].unique()

# ----------------------------
# 2️⃣ Filtrar historial hasta 2019-12-31 para los productos a predecir
# ----------------------------
historico = df_ag[
    (df_ag['item_id'].isin(ids_pred)) &
    (df_ag['timestamp'] <= '2019-12-31')
].copy()

# ----------------------------
# 3️⃣ Convertir historial a TimeSeriesDataFrame
# ----------------------------
required_columns = ['item_id', 'timestamp', 'target'] + features
historico = historico[required_columns].copy()

ts_data = TimeSeriesDataFrame.from_data_frame(
    historico,
    id_column='item_id',
    timestamp_column='timestamp'
)

# ----------------------------
# 4️⃣ Generar fechas futuras para el horizonte de predicción
# ----------------------------
df_future = predictor.make_future_data_frame(ts_data)

# ----------------------------
# 5️⃣ Preparar covariables para fechas futuras
# ----------------------------
# Extraemos las features que son estáticas por item_id (sin target ni timestamp)
df_static = df_ag[df_ag['item_id'].isin(ids_pred)] \
    .drop(columns=['target', 'timestamp']) \
    .drop_duplicates(subset='item_id')

# Combinamos con fechas futuras
df_future_covs = df_future.merge(df_static, on='item_id', how='left')

# ----------------------------
# 6️⃣ Convertimos covariables a TimeSeriesDataFrame
# ----------------------------
known_cov_tsdf = TimeSeriesDataFrame.from_data_frame(
    df_future_covs,
    id_column='item_id',
    timestamp_column='timestamp'
)

# ----------------------------
# 7️⃣ Predecimos
# ----------------------------
forecast_df = predictor.predict(
    data=ts_data,
    known_covariates=known_cov_tsdf
)

# ----------------------------
# 8️⃣ Filtramos resultados para 2020-02-01 y exportamos
# ----------------------------
forecast_202002 = forecast_df.reset_index().query("timestamp == '2020-02-29'")
forecast_202002[['item_id', 'mean']] \
    .rename(columns={'item_id': 'product_id', 'mean': 'tn'}) \
    .to_csv('pronostico_productos_202002_enriquecido.csv', index=False)

print("✅ Predicciones generadas y guardadas en pronostico_productos_202002_enriquecido.csv")


data with frequency 'MS' has been resampled to frequency 'M'.
data with frequency 'MS' has been resampled to frequency 'M'.
Model not specified in predict, will default to the model with the best validation score: DirectTabular


✅ Predicciones generadas y guardadas en pronostico_productos_202002_enriquecido.csv


## ✔️ Conclusión

El pipeline está listo para experimentar con variantes de features, ajustes de modelos y predicción automática de ventas por producto para cualquier horizonte.

- Podés repetir el pipeline para otros períodos o conjuntos de productos.
- Se pueden agregar rolling windows, rezagos, estacionalidad, o features macroeconómicos adicionales para mejorar el modelo.

¿Listo para modelar y experimentar?


## 📈 Visualización y métricas de resultados por producto

Analizamos la performance del modelo por producto, visualizando ventas reales vs. pronosticadas y calculando métricas de error relevantes.


### 📉 Celda de código – Graficar reales vs. pronosticados

In [68]:

# Suponiendo que forecast contiene todas las predicciones para todos los períodos
# Y df_ag contiene los reales
def plot_real_vs_pred(product_id, df_real, df_forecast):
    real = df_real[df_real['item_id'] == product_id].set_index('timestamp')
    pred = df_forecast[df_forecast.index.get_level_values('item_id') == product_id]
    pred = pred.set_index('timestamp')

    plt.figure(figsize=(12,5))
    plt.plot(real['target'], label='Real', marker='o')
    plt.plot(pred['mean'], label='Pronosticado', marker='x', linestyle='--')
    plt.title(f'Producto {product_id} - Real vs Pronóstico')
    plt.xlabel('Fecha')
    plt.ylabel('Toneladas')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Ejemplo de uso: graficar para los primeros 3 productos
for pid in df_ag['item_id'].unique()[:3]:
    plot_real_vs_pred(pid, df_ag, forecast)


NameError: name 'forecast' is not defined

### 🧮 Celda de código – Métricas de error por producto

In [73]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# ✅ Conversión correcta
df_real = pd.DataFrame(ts_data).reset_index()
df_forecast = forecast_df.reset_index()

metrics = []
for pid in df_real['item_id'].unique():
    real = df_real[df_real['item_id'] == pid].set_index('timestamp')
    pred = df_forecast[df_forecast['item_id'] == pid].set_index('timestamp')

    comunes = real.index.intersection(pred.index)
    if len(comunes) > 0:
        y_true = real.loc[comunes, 'target']
        y_pred = pred.loc[comunes, 'mean']
        mae = mean_absolute_error(y_true, y_pred)
        mape = mean_absolute_percentage_error(y_true, y_pred)
        wape = y_true.sub(y_pred).abs().sum() / y_true.sum()
        metrics.append({'product_id': pid, 'MAE': mae, 'MAPE': mape, 'WAPE': wape})

if not metrics:
    print("⚠️ No se encontraron productos con predicciones para los timestamps esperados.")
else:
    df_metrics = pd.DataFrame(metrics)
    print(df_metrics.sort_values('WAPE', ascending=False).head())


⚠️ No se encontraron productos con predicciones para los timestamps esperados.


## 🌟 Importancia de características y LeaderBoard de modelos

A continuación analizamos la importancia de variables del modelo seleccionado y visualizamos el desempeño de los modelos entrenados en AutoGluon.


In [77]:
# Importancia de características del mejor modelo
feature_importance = predictor.feature_importance(df_ag)
display(feature_importance)

# LeaderBoard de modelos entrenados
lb = predictor.leaderboard(df_ag, silent=True)
display(lb)


data with frequency 'IRREG' has been resampled to frequency 'M'.


ValueError: Cannot reserve last 2 time steps for evaluation in some time series in data. Please make sure that data includes both historical and future data, and thatall time series have length > prediction_length (at least 3)